# Clasificador binario
Vamos a utilizar Spacy y scikit-learn para clasificar con conjunto de tweets en español como positivos/negativos (análisis de sentimientos)

## Carga y preparación de los datos

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

# Leemos los datos
df = pd.read_csv('tweets_all.csv', index_col=None)

df.head()

In [ ]:
df.info()

In [ ]:
df.polarity.value_counts()

Tenemos 1514 tweets, de los cuales hay 637 positivos y 474 negativos. El resto son neutros o no tienen polaridad clasificada.
Vamos a entrenar sólo con los positivos y negativos para utilizar un clasificador binario

In [ ]:
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

In [ ]:
df.polarity.value_counts()

In [ ]:
df.info()

In [ ]:
df.sample(5)

Quitamos las columnas que no usamos.

## Limpieza de texto
Hacemos un pequeño pre-procesado del texto antes de extraer las características:  
- Quitamos las menciones y las URL del texto porque no aportan valor para el análisis de sentimientos.
- Los hashtag sí que pueden aportar valor así que simplemente quitamos el #.
- Quitamos los signos de puntuación y palabras menores de 3 caracteres.
- Por último quitamos todos los símbolos de puntuación del texto (que forman parte de un token).
- Lematizamos el texto y lo guardamos en otra columna para comparar resultados del clasificador. 

In [ ]:
import re, string, spacy
nlp=spacy.load('es_core_news_md')

In [ ]:
#lista de stop-words específicos de nuestro corpus (aproximación)
stop_words = ['unos', 'unas', 'algún', 'alguna', 'algunos', 'algunas', 'ese', 'eso', 'así']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, lemas=False):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Si lemas=True extraemos el lema, si no dejamos en minúsculas solamente.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lemma_.lower() if lemas else tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', tok) for tok in tokens if not (tok in stop_words) and len(tok)>2]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text
    

Probamos el funcionamiento de estas funciones sobre un tweet de ejemplo:

In [ ]:
type(df.content[702])

In [ ]:
type(clean_text(df.content[702]))

In [ ]:
print('Original:\n',df.content[702])
print('\nLimpiado:\n',clean_text(df.content[702]))
print('\nLematizado:\n',clean_text(df.content[702], lemas=True))

Aplicamos limpieza a todos los tweets del DataFrame y creamos columna nueva con los lemas

In [ ]:
df["limpio"]=df['content'].apply(clean_text)

In [ ]:
#Quitamos tweets vacíos después de la limpieza
df=df[df.limpio!='']

In [ ]:
df.info()

In [ ]:
df["lemas"]=df.content.apply(clean_text, lemas=True)

In [ ]:
df.head()

In [ ]:
#Contamos el nº de palabras por tweet
df['words'] = [len(t.split(' ')) for t in df.limpio]

In [ ]:
df['words'] = df['limpio'].apply(lambda t: len(t.split(' '))) #igual que la anterior

In [ ]:
df.describe()

### Clasificador
Vamos a usar la librería scikit-learn para aplicar un clasificador binario sobre la polaridad usando una extracción de características Bag-of-Words (BoW)

Primero dividimos en conjunto de entrenamiento y test.

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test, X_train_lema, X_test_lema = train_test_split(df['limpio'], 
                                                    df['polarity'],
                                                    df['lemas'],
                                                    test_size=0.3,
                                                    random_state=0
                                                    )

In [ ]:
print('Primera entrada de train:\n', X_train.iloc[0])
print('Polaridad:', y_train.iloc[0])
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
y_train.shape

In [ ]:
type(X_train)

In [ ]:
X_train.head(5)

In [ ]:
type(y_train)

In [ ]:
y_train.head(5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# aprendemos el modelo CountVectorizer sobre el conjunto de train
vect = CountVectorizer()

X_train_vectorized = vect.fit_transform(X_train)
X_train_vectorized

Vemos el número de términos distintos que tiene el diccionario:

In [ ]:
len(vect.get_feature_names_out())

In [ ]:
np.random.choice(vect.get_feature_names_out(), 5, replace=False)

### Entrenamiento del modelo
Vamos a probar un clasificador Logistic Regression de scikit-learn para entrenar nuestro modelo

In [ ]:
from sklearn.linear_model import LogisticRegression

modelLR = LogisticRegression(solver='liblinear')
#Entrenamos el modelo con el conjunto de train
modelLR.fit(X_train_vectorized, y_train)

In [ ]:
prediccion_train = modelLR.predict(X_train_vectorized)
pd.DataFrame({
    'texto':X_train,
    'polaridad':y_train,
    'predicción':prediccion_train
}).sample(10)

In [ ]:
from sklearn.metrics import accuracy_score

print('Exactitud: ', accuracy_score(y_train, prediccion_train))

### Verificación del modelo
Para ver el rendimiento del modelo usamos el conjunto de test. Primero transformamos el conjunto de test a su matriz BoW mediante el vectorizador aprendido en TRAIN y aplicamos el modelo entrenado:

In [ ]:
# Predecimos sobre el conjunto de test
X_test_vectorized = vect.transform(X_test)
X_test_vectorized.shape

In [ ]:
X_test_vectorized

In [ ]:
prediccion = modelLR.predict(X_test_vectorized)

In [ ]:
prediccion.shape

Vemos el resultado de la predicción y calculamos su precisión con distintas métricas.  
Ejemplo de predicción de algunas muestras:

In [ ]:
pd.DataFrame({'texto':X_test, 'polaridad':y_test, 'predicción':prediccion}).sample(10)

### Exactitud del modelo
(# predicciones correctas / Total de muestras)

In [ ]:
from sklearn.metrics import accuracy_score

print('Exactitud: ', accuracy_score(y_test, prediccion))

Matriz de confusión (predicción -columnas- frente a etiquetas reales -filas-)

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test, prediccion)
pd.DataFrame(cm, index=('N_true','P_true'), columns=('N_pred','P_pred'))
#filas: True, columnas: Prediction

Podemos ver un informe más completo del clasificador con la métrica `classification_report`:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, prediccion))

`precision` es la precisión: TP/(TP+FP) (probabilidad de que un positivo detectado sea un verdadero positivo)  
`recall` es la sensibilidad: TP/(TP+FN) (probabilidad de predicción positiva para una muestra positiva)  
`support` indica el número de muestras en cada clase en el conjunto de test (suma por filas en la matriz de confusión)

### Área bajo la curva ROC:  
Para calcular el área bajo la curva ROC (AUC) es necesario obtener la probabilidad de salida del clasificador con `predict_proba`

In [ ]:
from sklearn.metrics import roc_auc_score

prediccion_prob = modelLR.predict_proba(vect.transform(X_test))
#la primera columna corresponde a la etiqueta 'N'
#Es necesario convertir los True Labels a un array lógico (1 para etiqueta N)
roc_auc_score((y_test=='N'), prediccion_prob[:,0])

### Veamos qué palabras son las más relevantes en el modelo

In [ ]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(vect.get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelLR.coef_[0].argsort()

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Vemos que Naive Bayes mejora al modelo de Regresión Logística

### Optimización del código
Combinamos la extracción de características y clasificación en un `pipeline` de scikit-learn (https://scikit-learn.org/stable/modules/compose.html#)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

vect = CountVectorizer()
modelLR = LogisticRegression(solver='liblinear')

modelo = make_pipeline(vect, modelLR)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

In [ ]:
len(modelo['countvectorizer'].get_feature_names_out())

In [ ]:
modelo

In [ ]:
modelo[0]

Equivale a:

In [ ]:
vect

In [ ]:
modelo[0].get_feature_names_out()

In [ ]:
vect.get_feature_names_out()

In [ ]:
modelo[1]

Equivale a:

In [ ]:
modelLR

In [ ]:
modelo[1].coef_[0]

In [ ]:
modelLR.coef_[0]

In [ ]:
# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print(classification_report(y_test, prediccion))

## Otros modelos
Probamos con los modelos Naïve Bayes y un SVM lineal para ver si mejora

In [ ]:
from sklearn.naive_bayes import MultinomialNB

modelNB = MultinomialNB()

#Entrenamos el modelo con el conjunto de train
modelNB.fit(X_train_vectorized, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelNB.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
from sklearn.linear_model import SGDClassifier

modelSVM = SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5)

#Entrenamos el modelo con el conjunto de train
modelSVM.fit(X_train_vectorized, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelSVM.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

## Modelo con vectores TF-IDF
Cambiamos el vectorizador para ver si hay mejoría

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()

modelo = make_pipeline(vect, modelLR)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print(classification_report(y_test, prediccion))

In [ ]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(modelo['tfidfvectorizer'].get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelo['logisticregression'].coef_[0].argsort()

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

### Otros modelos con TF-IDF
Repetimos con los modelos NB y SVM

In [ ]:
modelo = make_pipeline(vect, modelNB)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print(classification_report(y_test, prediccion))

In [ ]:
modelo = make_pipeline(vect, modelSVM)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print(classification_report(y_test, prediccion))

## Modelos con texto lematizado
Repetimos con el texto lematizado para ver si hay diferencias

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
# Asignamos un 70% a training y un 30% a test
X_train_lema, X_test_lema, y_train, y_test = train_test_split(df['lemas'], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
X_train_lema.head(5)

### Modelos BoW con texto lematizado

In [ ]:
#vectorizamos
vect = CountVectorizer()

X_train_vectorized = vect.fit_transform(X_train_lema)
X_test_vectorized = vect.transform(X_test_lema)
X_train_vectorized.shape

In [ ]:
len(modelo[0].get_feature_names_out())

In [ ]:
#Modelo BoW-LR
modelLR.fit(X_train_vectorized, y_train)
prediccion = modelLR.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
#Modelo BoW-NB
modelNB.fit(X_train_vectorized, y_train)
prediccion = modelNB.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
#Modelo BoW-SVM
modelSVM.fit(X_train_vectorized, y_train)
prediccion = modelSVM.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(vect.get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelLR.coef_[0].argsort()

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

### Modelos TF-IDF con texto lematizado

In [ ]:
#vectorizamos
vect = TfidfVectorizer()

X_train_vectorized = vect.fit_transform(X_train_lema)
X_test_vectorized = vect.transform(X_test_lema)
X_train_vectorized.shape

In [ ]:
#Modelo BoW-LR
modelLR.fit(X_train_vectorized, y_train)
prediccion = modelLR.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
#Modelo BoW-NB
modelNB.fit(X_train_vectorized, y_train)
prediccion = modelNB.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
#Modelo BoW-SVM
modelSVM.fit(X_train_vectorized, y_train)
prediccion = modelSVM.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

In [ ]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(vect.get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelLR.coef_[0].argsort()

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

## Modelos n-gramas

In [ ]:
#vectorizamos
vect = CountVectorizer(ngram_range=(1, 2), min_df=2)

X_train_vectorized = vect.fit_transform(X_train_lema)
X_test_vectorized = vect.transform(X_test_lema)
X_train_vectorized.shape

In [ ]:
np.random.choice(vect.get_feature_names_out(), 5, replace=False)

In [ ]:
#Entrenamos los 3 clasificadores con las características BoW-bigramas
modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM)]
for m, clf in modelos:
    clf.fit(X_train_vectorized, y_train)
    prediccion = clf.predict(X_test_vectorized)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

Faltaría probar otras combinaciones:  
- TF-IDF con bigramas
- Bigramas con texto sin lematizar
- Reducción del vocabulario con `min_df` y `max_df`

In [ ]:
#Quitamos palabras presentes en más del 10% de documentos
vect = TfidfVectorizer(max_df=0.1)
X_train_vectorized = vect.fit_transform(X_train_lema)
X_test_vectorized = vect.transform(X_test_lema)
print(len(vect.get_feature_names_out()))

In [ ]:
vect.stop_words_

In [ ]:
#Entrenamos los 3 clasificadores con las características BoW quitando stop-words
modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM)]
for m, clf in modelos:
    clf.fit(X_train_vectorized, y_train)
    prediccion = clf.predict(X_test_vectorized)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

### Modelo sin preprocesado
Por comparar probamos un modelo Bow-LR sin pre-procesar el texto

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
#vectorizamos
vect = CountVectorizer()

X_train_vectorized = vect.fit_transform(X_train)
X_test_vectorized = vect.transform(X_test)
X_train_vectorized.shape

In [ ]:
#Modelo BoW-LR
modelLR.fit(X_train_vectorized, y_train)
prediccion = modelLR.predict(X_test_vectorized)
print(classification_report(y_test, prediccion))

Baja la exactitud del modelo de un 76% a un 72%

## Modelo con word embeddings
Ahora vamos a usar como espacio de características los *word vectors* de las palabras de nuestro corpus.  
Como cada palabra tiene un vector de longitud fija, tenemos que obtener un único vector como promedio de todas las palabras del tweet.  
En spaCy, el vector de cada palabra es el atributo `vector`.  
El atributo `vector` del objeto `Doc` del texto procesado en spaCy contiene el vector promedio de todos los tokens.

Vemos el tamaño del vector del modelo `Spacy`

In [ ]:
nlp.vocab.vectors_length

Es el tamaño del vector de cada token

In [ ]:
doc=nlp(df.content[1])
doc[1].vector.shape

In [ ]:
df.content[1]

Que coincide con el tamaño del vector del documento entero:

In [ ]:
doc.vector.shape

Este vector corresponde al promedio de los vectores de todos los tokens del documento que tienen un vector definido en `spaCy`

In [ ]:
#Cada vector tiene un tamaño de 50, por tanto hay que crear una matriz de
#tamaño (nº documentos,50) para guardar el promedio de los vectores de cada tweet
#y guardar en cada fila el correspondiente vector promedio
word_embeddings=np.zeros((len(df), nlp.vocab.vectors_length))

In [ ]:
word_embeddings.shape

In [ ]:
#Spacy ya calcula el promedio de los vectores de un documento en Doc.vector
vectors = [nlp(tweet).vector for tweet in df.limpio]
for i,vector in enumerate(vectors):
    word_embeddings[i,:]=vector

In [ ]:
type(word_embeddings)

Generamos los conjuntos de entrenamiento con word embeddings de cada tweet y volvemos a aplicar los mismos clasificadores de antes.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(word_embeddings, 
                                                    df['polarity'], 
                                                    test_size=0.3,
                                                    random_state=0)

Aplicamos un clasificador a esta matriz de características. En este caso la matriz conviene valores decimales, por lo que el clasificador `MultinomialNB` se tiene que sustituir por un `GaussianNB` para usar un modelo Naïve Bayes, pero también podemos probar otros modelos más complejos (p. ej. un SVM con un kernel RFB)

In [ ]:
X_train.shape

In [ ]:
#entrenamos clasificadores con modelos word embeddings
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', GaussianNB()),
           ('Linear SVM', modelSVM),
           ('RFB SVM', SVC(gamma='scale', C=2))]

for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')


Los modelos con word embedding promediado para todo el tweet funcionan un poco peor que modelos más simples (BoW, TF-IDF). Para usar word embeddings conviene irse a un modelo secuencial (por ejemplo con LSTM), para lo que es necesario entrenar con un conjunto de datos mucho mayor.